## LLM Agents that can Code

The notebook here has the mere purpose of describing and explaining the code we're going to use, not of actually run the code.

The reason for this is that in this section we're going to ask our agents to write code and execute it for us. Therefore we'll execute scripts when it comes to code creation by agents. This has also the advantage that our code can be deployed to a cloud environment.

In [ ]:
import pandas as pd

In [ ]:
api_key = pd.read_csv("~/tmp/chat_gpt/autogen_agent_1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

In [ ]:
llm_config = {
    "model": "gpt-4o",
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

### Command Line Executor

The __Command Line Executor__ is a tool from autogen that allows us to execute code locally on our laptop.

This tool offers two differnt solutions for executing code: 

1. __Executing code in a Docker container:__ Inside Docker a virtualized version of a python machine will be created to execute the code. This version is more secure because the code always gets executed within a virtual machine. If the LLM generates malicious code it won't have a direct effect on our machine. 

2. __Executing code locally on your machine:__ This is the version (or solution we'll use here). We will be generate only simple code that and review it each time before execution. In addition we will specify that if the code hangs and nothing happens during 60 seconds we will interrupt it. All code should be executed and provide results in the `coding` folder.

In [ ]:
# importing from autogen what's needed for setting up a local executor environment for our agents
from autogen.code_utils import create_virtual_env
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

In [ ]:
# we're going to ask the agent to execute its own virtual environment
venv_dir = ".env_llm"
venv_context = create_virtual_env(venv_dir) # imported from autogen.code_utils

# we don't see what the agent is doing while he's creating code so we set a timeout in case he got stuck somewhere, like in a loop
executor = LocalCommandLineCodeExecutor(
    virtual_env_context=venv_context,
    timeout=200,
    work_dir="coding",
)
# printing out the Python used by the agent
print(
    executor.execute_code_blocks(code_blocks=[CodeBlock(language="python", code="import sys; print(sys.executable)")])
)

### Code Writer Agent

For the task of writing code we'll need two agents.

> __Agent no. 1: Code Writer Agent.__ The **code_writer_agent** is our engineer who will get a task, think about how to solve the task and rely on ChatGPT to generate python code to accomplish the requested task. This agent is of the type `AssistantAgent` which is designed to write code.

In [ ]:
from autogen import AssistantAgent

In [ ]:
# Defining the Code Writer Agent
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False, # agent writes code but doesn't execute it
    human_input_mode="NEVER", # human input never necessary here
)
code_writer_agent

The **Code Writer Agent** we created does not execute code (`code_execution_config=False`) and it doesn't need a system prompt message to tell it about his role. It has a default system prompt message that is designed to suggest the use of python code when it can help to solve the task.

__Default system prompt message of the AssistantAgent:__

> You are a helpful AI assistant.<br/>
Solve tasks using your coding and language skills.<br/>
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.<br/>
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.<br/>
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.<br/>
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.<br/>
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.<br/>
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.<br/>
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.<br/>
Reply "TERMINATE" in the end when everything is done.<br/>

If we define a system prompt for this agent we overwrite the existing one.

In [ ]:
# let's have a look at the default system prompt message of this type of agent
code_writer_agent_system_message = code_writer_agent.system_message
print(code_writer_agent_system_message)

### Code Executor Agent

> __Agent no. 2: Code Executor Agent.__ The **code_executor_agent** does not use an LLM (`llm_config=False`), instead it will just execute the code it was provided with. The code execution will happen locally in the `coding` folder and the result of that code execution will be sent back as a reply.<br/><br/>
> The **code_executor_agent** needs a `code_execution_config` to be able to execute code. We will provide it with the the configuration we defined earlier and that was called `executor` (`code_execution_config={"executor": executor}`).<br/><br/>
> In addition to this we will give the **code_executor_agent** a default auto reply that it will use if the user does not have any additional details to provide.<br/><br/>
> Each time this agent will execute some code it will request feedback from the user If the user does not provide any feedback, it will use the auto reply.

In [ ]:
# the code_executor_agent is a ConversableAgent
from autogen import ConversableAgent

In [ ]:
# Defining the code executor agent
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)
code_executor_agent

### Defining the Coding Task

We will now ask our agents to perform a task that requires coding. 

1. The task will be provided to the Code Writer agent by the Code Executor agent

2. The Code Writer agent will then propose a code that should fulfill the task

3. The Code Executor agent will then execute that code and report back the resutl to the Code Writer.

There might be several exchanges between these two agents until they accomplish the task.  

For example, if a code error is encountered, the error will be reported to the Code Writer who'll propose a corrected version of the code.

Let's prepare a simple task to test our code execution scheme. We want our agents to generate a plot that shows the price evolution of two assets, NVDA and BTC for the last 4 years in a way that makes it easy for us to compare them. We will also add the 50 weeks moving average.

In [ ]:
import datetime

today = datetime.datetime.now().date()

message = f"Today is {today}. "\
"Create a plot showing the normalized price of NVDA and BTC-USD for the last 4 years "\
"with their 50 weeks moving average. "\
"Make sure the code is in a markdown code block, save the figure"\
" to a file asset_analysis.png and show it. Provide all the code necessary in a single python bloc. "\
"Re-provide the code block that needs to be executed with each of your messages. "\
"If python packages are necessary to execute the code, provide a markdown "\
"sh block with only the command necessary to install them and no comments."

During the execution, the executor might (and will most likely will especially if it is the first time) encounter some errors. For example, if you did not install the modules required by the code writer, you will get an error. You will most likely have to go install these packages yourself in your virtual environment, as we did when we installed `openai`. The agents might also just install the missing moduels themselves.

For this task, the code writer will most likely require `matplotlib` and maybe (not necessarily) `pandas`. The LLMs will however tell you how to install missing packages as they will be able to interprete the output of the code and tell you how to fix it. 

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=message
)

The code will now execute.

Take the time to observe the interaction between the agents. Before executing the code provided by the Code Writer, you will be requested to provide feedback. That is an important safety feature. Always review the code written by an agent before executing it, you never know, there might be some malicious content. You can also provide feedback if you take something is wrong with the code. If not, you can just press Enter to let the Code Executor go ahead and execute the code.